<a href="https://colab.research.google.com/github/drewwint/ABCD_subset_project/blob/main/faster_hitting_time_demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speeding up hitting time
By: Drew E. Winters, PhD

I updated the function to include parallel processing to speed up calculation for larger datasets.
- This successfully reduced time considerably and produces ~ the same results with only minor differnces that is very unlikely to impact hitting time estimation in any significant way.


This was based on the following citation
- Rezaeinia, P., Fairley, K., Pal, P., Meyer, F. G., & Carter, R. M. (2020). Identifying brain network topology changes in task processes and psychiatric disorders. Network Neuroscience, 4(1), 257-273.



## Origional function

In [14]:
import time

## Old version
def hitting_matrix(correlation_matrix):
  start_time = time.perf_counter()
  correlation_matrix = abs(correlation_matrix)  ## added this to only retain absolute values.
  L = np.size(correlation_matrix,axis = 0)
  A_matrix = np.array(correlation_matrix)
  D_matrix = np.zeros((L,L))
  for i in range(L):
      D_matrix[i,i] = np.sum(A_matrix[i]) # loop unnecessary - just sum the columns

  d_max = np.max(D_matrix)

  for j in range(L):
      if np.max(A_matrix[j,:]) < .05:
          A_matrix[j,j] = d_max - D_matrix[j,j] # no need to make a matrix with a values on the diagonal - just a list

  D_matrix = np.zeros((L,L))
  D_inv = np.zeros((L,L))
  D_sqrt = np.zeros((L,L))
  D_sqrt_inv = np.zeros((L,L))
  for i in range(L):
      D_matrix[i,i] = np.sum(A_matrix[i])
      D_inv[i,i] = 1./D_matrix[i,i]
      D_sqrt[i,i] = np.sqrt(D_matrix[i,i])
      D_sqrt_inv[i,i] = 1./D_sqrt[i,i]

  p_matrix = np.dot(D_inv, A_matrix)

  # Graph Laplacian
  eye_matrix = np.eye(L,L)
  eye_P = eye_matrix - p_matrix

  G_Lap = np.dot(D_sqrt,eye_P)
  G_Lap_n = np.dot(G_Lap, D_sqrt_inv)

  [eig_val, eig_vec] = np.linalg.eigh(G_Lap_n)
  H = np.zeros((L,L))
  d = np.sum(D_matrix)
  for i in range(L):
      for j in range(L):
          deg_i = D_matrix[i,i]
          deg_j = D_matrix[j,j]
          for k in range(L):
              if eig_val[k] != min(eig_val):
                  t_i = (eig_vec[i,k]*eig_vec[i,k])/deg_i
                  t_j = (eig_vec[j,k]*eig_vec[j,k])/deg_j
                  t_ij = (eig_vec[i,k]*eig_vec[j,k])/np.sqrt(deg_i*deg_j)
                  H[i,j] = H[i,j] + d*(1./(eig_val[k]))*(t_j-t_ij)

  H = np.transpose(H)
  end_time = time.perf_counter()
  print(f"origional total time: {end_time - start_time:.2f} seconds")
  return H

## Speed up function

In [15]:

from joblib import Parallel, delayed
import numpy as np

def hitting_matrix_p2(correlation_matrix):
    start_time = time.perf_counter()
    correlation_matrix = np.array(abs(correlation_matrix))  # Ensure absolute values
    np.fill_diagonal(correlation_matrix, 0)  # Set diagonal to 0

    L = correlation_matrix.shape[0]
    A_matrix = correlation_matrix.copy()

    # Degree matrix
    row_sums = A_matrix.sum(axis=1) # instead of d_matrix loop we sum columns without the loop
    d_max = row_sums.max()

    # Ensure graph connectivity
    for j in range(L):
      if np.max(A_matrix[j,:]) < .05:
          A_matrix[j,j] = d_max - row_sums[j]

    row_sums = A_matrix.sum(axis=1)  # Recalculate after adjustment
    D_inv = np.diag(1.0 / row_sums)
    D_sqrt = np.diag(np.sqrt(row_sums))
    D_sqrt_inv = np.diag(1.0 / np.sqrt(row_sums))

    # Transition probability matrix and Graph Laplacian
    p_matrix = D_inv @ A_matrix
    eye_P = np.eye(L) - p_matrix
    G_Lap_n = D_sqrt @ eye_P @ D_sqrt_inv

    # Eigen decomposition
    eig_val, eig_vec = np.linalg.eigh(G_Lap_n)

    # Precompute reusable quantities
    eig_val_nonzero = eig_val[eig_val > eig_val.min()]
    eig_vec_squared = eig_vec ** 2
    d_total = row_sums.sum()

    def compute_H_row(i):
        H_row = np.zeros(L)
        deg_i = row_sums[i]
        for j in range(L):
            deg_j = row_sums[j]
            t_ij = (
                eig_vec_squared[i, eig_val > eig_val.min()] / deg_i
                - eig_vec[i, eig_val > eig_val.min()]
                * eig_vec[j, eig_val > eig_val.min()]
                / np.sqrt(deg_i * deg_j)
            )
            H_row[j] = np.sum(d_total * t_ij / eig_val_nonzero)
        return H_row

    # Parallelize computation of rows
    with Parallel(n_jobs=-1, backend="loky") as parallel:
        H = np.array(parallel(delayed(compute_H_row)(i) for i in range(L)))
    end_time = time.perf_counter()
    print(f"faster function total time: {end_time - start_time:.2f} seconds")
    return H



# Comparing the functions

## Simulating data

In [16]:
np.random.seed(1979)
N = 216

# Create a symmetric positive semi-definite correlation-like matrix
A = np.random.rand(N, N)
corr_matrix = (A + A.T) / 2  # make symmetric
np.fill_diagonal(corr_matrix, 1.0)


## Comparing output and time to estimate

In [17]:

# Run both functions
H1 = hitting_matrix(corr_matrix)
H2 = hitting_matrix_p2(corr_matrix)

# Compare outputs
abs_diff = np.abs(H1 - H2)
row_mean_diff = np.abs(H1.mean(axis=1) - H2.mean(axis=1))
col_mean_diff = np.abs(H1.mean(axis=0) - H2.mean(axis=0))

print("\n=== Comparison Summary ===")
print(f"Matrix Mean Absolute Difference: {np.mean(abs_diff):.4e}")
print(f"Row Mean Absolute Difference:   {np.mean(row_mean_diff):.4e}")
print(f"Col Mean Absolute Difference:   {np.mean(col_mean_diff):.4e}")

origional total time: 225.56 seconds
faster function total time: 4.59 seconds

=== Comparison Summary ===
Matrix Mean Absolute Difference: 1.9927e+00
Row Mean Absolute Difference:   1.9927e+00
Col Mean Absolute Difference:   1.9927e+00


# Conclusion
The new function takes considerably less time to estimate
- 4.59 seconds in the new function VS 225.56 seconds in the origional
- Making the new function ~ 98% faster than the origional

On average the faster function estimates about 2 less hops, which is consistent across all cells in the hitting matrix
- This acts as a scaling back of about 2 hops consistently
- Thus it retains the relative hitting time values
- Therefore it is consistent and will not bias results one way or another

It is advisable to use the new function for faster hitting time estimation